# Thực nghiệm thuật toán Random Forest trên tập dataset 2

<h2> Table of content </h2>
<div class ="alert alert-block alert-info" style="margin-top: 20px">
  <ol>
    <li><a href = "#1"> Tạo tập dữ liệu </a></li>
    <li><a href = "#2"> Thử nghiệm mô hình mặc định </a></li>
    <li><a href = "#tuning"> Tinh chỉnh siêu tham số  </a></li>
  </ol>
</div>

<h3 id = "1"> 1. Tạo tập dữ liệu </h3>

In [1]:
import pandas as pd

from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

In [2]:
data_train = pd.read_csv("Data2_train.csv")
data_test = pd.read_csv("Data2_test.csv")

data_train = data_train.drop(data_train.columns[0], axis = 1)
data_test = data_test.drop(data_test.columns[0], axis = 1)

data_train.head()

,GpsProvider,Market/Regular,vehicle_no,Origin_Location,Destination_Location,Planned_ETA,ontime,OriginLocation_Code,DestinationLocation_Code,trip_start_date,TRANSPORTATION_DISTANCE_IN_KM,vehicleType,customerID,supplierID,Material Shipped,Org_lat,Org_lon,Des_lat,Des_lon
0,3,0,136,23,383,173,0.0,127,169,157,80.00,34,15,92,764,16.560192,80.792293,23.704770,72.539096
1,3,0,1467,93,135,290,0.0,111,224,263,43.00,34,15,95,692,28.722966,77.345364,28.476329,77.733763
2,6,0,1188,82,92,221,0.0,127,200,200,53.90,34,15,28,229,16.560192,80.792293,12.730174,77.956202
3,3,0,559,82,70,103,0.0,153,159,93,31.00,34,15,28,271,12.992700,77.803841,12.777875,77.642276
4,3,0,909,37,161,172,0.0,76,335,154,360.15,36,15,26,578,18.970774,72.808712,21.606548,73.009534


Dữ liệu cần dự đoán ở đây là cột ontime

In [3]:
X_train = data_train.iloc[:, data_train.columns != "ontime"].values
y_train = data_train["ontime"].values

X_test = data_test.iloc[:, data_test.columns != "ontime"].values
y_test = data_test["ontime"].values

<h3 id = "2"> 2. Thử nghiệm mô hình mặc định </h3>

In [9]:
model = RandomForestClassifier()
model.fit(X_train, y_train)
predict = model.predict(X_test)
print(classification_report(y_test, predict))

              precision    recall  f1-score   support

         0.0       0.95      0.95      0.95       821
         1.0       0.74      0.73      0.73       161

    accuracy                           0.91       982
   macro avg       0.84      0.84      0.84       982
weighted avg       0.91      0.91      0.91       982



<h3 id = "tuning"> 3. Tinh chỉnh siêu tham số </h3>

Các siêu tham số trong mô hình Random Forest bao gồm:
<li> <b> Number of estimators</b>: Đây là số lượng cây quyết định được sử dụng trong mô hình. Một số lượng cây quyết định quá ít có thể dẫn đến việc mô hình không đủ tốt, trong khi một số lượng quá nhiều có thể dẫn đến việc mô hình bị quá khớp.
<li> <b> Maximum depth </b>: Đây là độ sâu tối đa của mỗi cây quyết định. Một độ sâu cây quá lớn có thể dẫn đến việc mô hình bị quá khớp, trong khi một độ sâu quá thấp có thể dẫn đến việc mô hình không đủ tốt.
<li> <b> Min samples leaf </b>: Đây là số lượng mẫu tối thiểu cần thiết để tạo một nút lá trong cây quyết định. Một giá trị quá thấp có thể dẫn đến việc mô hình bị quá khớp, trong khi một giá trị quá cao có thể dẫn đến việc mô hình không đủ tốt.

Siêu tham số được tinh chỉnh bằng phương pháp đánh giá chéo (cross validaition) với việc tập dữ liệu được chia làm 10 phần không chồng chéo và có kích thước bằng nhau. 
Ngoài ra thước đo cho mô hình em chọn là F1 - score

In [12]:
from sklearn.model_selection import GridSearchCV

model = RandomForestClassifier()
param_grid = {
    'n_estimators': [60, 80, 100],
    'max_depth': [3, 4, 5],
    'min_samples_leaf': [1, 2, 3],
}

grid_search = GridSearchCV(model, param_grid, cv=10, scoring= 'f1_micro')
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)

{'max_depth': 5, 'min_samples_leaf': 3, 'n_estimators': 60}


In [4]:
model = RandomForestClassifier(n_estimators=60, max_depth=5, min_samples_leaf=3)
model.fit(X_train, y_train)
predict = model.predict(X_test)
print(classification_report(y_test, predict))

              precision    recall  f1-score   support

         0.0       0.92      0.98      0.95       821
         1.0       0.84      0.56      0.67       161

    accuracy                           0.91       982
   macro avg       0.88      0.77      0.81       982
weighted avg       0.91      0.91      0.90       982



In [ ]:
import os
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

if not os.path.exists('images'):
    os.makedirs('images')

for i in range(len(model.estimators_)):
    fig = plt.figure(figsize=(25,20))
    _ = plot_tree(model.estimators_[i], filled=True)
    fig.savefig(f"images/decistion_tree_{i+1}.png")
    plt.close()